# [STARTER] Exercise - Building an AI Agent with Tools

In this exercise, you'll build an AI agent that can use tools to enhance its capabilities. You'll learn how to create an agent that can understand when to use tools, process their results, and maintain a coherent conversation.

## Challenge

Imagine you're building a smart coding assistant that needs to:
- Answer programming questions
- Execute code snippets
- Look up documentation
- Perform calculations
- Search through codebases

Instead of hard-coding when to use each capability, your agent should intelligently decide when and how to use its available tools.

## Setup
First, let's import the necessary libraries:

In [1]:
from pathlib import Path
import sys

nb_dir = Path(__file__).parent if "__file__" in globals() else Path.cwd()
parent = nb_dir.parent  # points to 03-building-agents
if str(parent) not in sys.path:
    sys.path.insert(0, str(parent))

In [2]:
from typing import List, Dict, Any
from dotenv import load_dotenv
from copy import deepcopy
import json

from lib.messages import UserMessage, SystemMessage, ToolMessage
from lib.tooling import tool
from lib.llm import LLM

## Understanding the Components

Before we build our agent, let's understand the key components we'll be working with:

- `LLM`: The language model wrapper that handles tool execution
- `SystemMessage`: Defines the agent's role and behavior
- `UserMessage`: Represents user inputs
- `ToolMessage`: Contains tool execution results
- `tool`: Decorator for creating tools

## Building the Agent Class

Your task is to create an Agent class that can:
1. Initialize with a specific role and set of tools
2. Process user messages
3. Decide when to use tools
4. Handle tool responses

In [3]:
class Agent:
    """An AI Agent that can use tools to help answer questions"""
    
    def __init__(
        self,
        role: str = "Personal Assistant",
        instructions: str = "Help users with any question",
        model: str = "gpt-4o-mini",
        temperature: float = 0.0,
        tools: List[Any] = None
    ):
        """Initialize the agent with its configuration and tools"""
        # TODO 1: Initialize the agent
        # Hint: 
        # - Load environment variables with load_dotenv()
        # - Store agent settings (role, instructions, etc.)
        # - Create an LLM instance with the provided tools
        self.role = role
        self.instructions = instructions
        self.model = model
        self.temperature = temperature
        self.tools = tools

        load_dotenv()

        self.llm = LLM(model=model, temperature=temperature, tools=tools)
        

    def invoke(self, user_message: str) -> str:
        """Process a user message and return a response"""
        # TODO 2: Set up the conversation
        # Hint:
        # - Create messages list with SystemMessage (role + instructions)
        # - Add UserMessage with user_message
        # - Get initial AI response using self.llm.invoke()
        messages = [
            SystemMessage(
                content=(
                    f"You're an AI Agent and your role is {self.role}"
                    f"Your instructions: {self.instructions}"
                )
            )
        ]
        messages.append(UserMessage(content=user_message))
        ai_message = self.llm.invoke(messages)
        messages.append(ai_message)

        # TODO 3: Handle tool calls if needed
        # Hint:
        # - Check if ai_message.tool_calls exists
        # - For each tool call:
        #   * Get function name and arguments
        #   * Execute tool and get result
        #   * Add result as ToolMessage
        # - Get final AI response
        while ai_message.tool_calls:
            for call in ai_message.tool_calls:
                function_name = call.function.name
                function_args = json.loads(call.function.arguments)
                tool_call_id = call.id

                tool = next((t for t in self.tools if t.name == function_name), None)
                if tool:
                    result = tool(**function_args)
                    messages.append(
                        ToolMessage(
                            content=json.dumps(result),
                            tool_call_id=tool_call_id,
                            name=function_name,
                        )
                    )
            ai_message = self.llm.invoke(messages)
            messages.append(ai_message)

        for m in messages:
            print(m)
        return ai_message.content

## Testing Your Agent

Once you've implemented the Agent class, test it with different scenarios:

1. Basic conversation without tools

In [4]:
agent = Agent(role="Coding Assistant")
response = agent.invoke("What is Python? Be concise")
print(response)

role='system' content="You're an AI Agent and your role is Coding AssistantYour instructions: Help users with any question"
role='user' content='What is Python? Be concise'
role='assistant' content='Python is a high-level, interpreted programming language known for its readability and simplicity. It supports multiple programming paradigms, including procedural, object-oriented, and functional programming. Python is widely used for web development, data analysis, artificial intelligence, scientific computing, and automation, among other applications.' tool_calls=None
Python is a high-level, interpreted programming language known for its readability and simplicity. It supports multiple programming paradigms, including procedural, object-oriented, and functional programming. Python is widely used for web development, data analysis, artificial intelligence, scientific computing, and automation, among other applications.


2. Create a calculator tool

In [5]:
@tool
def calculate(expression: str) -> float:
    """Evaluate a mathematical expression"""
    return eval(expression)

In [6]:
# Create an agent with the calculator tool
math_agent = Agent(
    role="Math Assistant",
    tools=[calculate]
)

In [7]:
response = math_agent.invoke("What is 23 * 45?")
print(response)

role='system' content="You're an AI Agent and your role is Math AssistantYour instructions: Help users with any question"
role='user' content='What is 23 * 45?'
role='assistant' content=None tool_calls=[ChatCompletionMessageToolCall(id='call_ePeL9iEViNuCgT13kate2Cd3', function=Function(arguments='{"expression":"23 * 45"}', name='calculate'), type='function')]
role='tool' content='1035' tool_call_id='call_ePeL9iEViNuCgT13kate2Cd3' name='calculate'
role='assistant' content='The result of \\( 23 \\times 45 \\) is 1035.' tool_calls=None
The result of \( 23 \times 45 \) is 1035.


In [8]:
# Test multiple tool usage
response = math_agent.invoke("If I multiply 3 by 5, what do I get? Then later add 7")
print(response)

role='system' content="You're an AI Agent and your role is Math AssistantYour instructions: Help users with any question"
role='user' content='If I multiply 3 by 5, what do I get? Then later add 7'
role='assistant' content=None tool_calls=[ChatCompletionMessageToolCall(id='call_QUOT66cbFX6VOQnhLIKmTSOM', function=Function(arguments='{"expression": "3 * 5"}', name='calculate'), type='function'), ChatCompletionMessageToolCall(id='call_UwkFJJnzbcnqXRzNPJukFcLw', function=Function(arguments='{"expression": "(3 * 5) + 7"}', name='calculate'), type='function')]
role='tool' content='15' tool_call_id='call_QUOT66cbFX6VOQnhLIKmTSOM' name='calculate'
role='tool' content='22' tool_call_id='call_UwkFJJnzbcnqXRzNPJukFcLw' name='calculate'
role='assistant' content='If you multiply 3 by 5, you get 15. If you then add 7 to that result, you get 22.' tool_calls=None
If you multiply 3 by 5, you get 15. If you then add 7 to that result, you get 22.


3. Create a data analyst

In [9]:
GAMES_DATA = [
    {"Game": "The Legend of Zelda: Breath of the Wild", "Platform": "Switch", "Score": 98},
    {"Game": "Super Mario Odyssey", "Platform": "Switch", "Score": 97},
    {"Game": "Metroid Prime", "Platform": "GameCube", "Score": 97},
    {"Game": "Super Smash Bros. Brawl", "Platform": "Wii", "Score": 93},
    {"Game": "Mario Kart 8 Deluxe", "Platform": "Switch", "Score": 92},
    {"Game": "Fire Emblem: Awakening", "Platform": "3DS", "Score": 92},
    {"Game": "Donkey Kong Country Returns", "Platform": "Wii", "Score": 87},
    {"Game": "Luigi's Mansion 3", "Platform": "Switch", "Score": 86},
    {"Game": "Pikmin 3", "Platform": "Wii U", "Score": 85},
    {"Game": "Animal Crossing: New Leaf", "Platform": "3DS", "Score": 88}
]

@tool
def get_games(num_games:int=1, top:bool=True) -> str:
    """
    Returns the top or bottom N games with highest or lowest scores.    
    args:
        num_games (int): Number of games to return (default is 1)
        top (bool): If True, return top games, otherwise return bottom (default is True)
    """
    # Sort the games list by Score
    # If top is True, descending order
    sorted_games = sorted(GAMES_DATA, key=lambda x: x['Score'], reverse=top)
    
    # Return the N games
    return sorted_games[:num_games]

In [10]:
from typing import List, Dict, Any, Optional
import math


def _median(nums: List[float]) -> float:
    s = sorted(nums)
    n = len(s)
    mid = n // 2
    if n % 2 == 1:
        return float(s[mid])
    return (s[mid - 1] + s[mid]) / 2.0

def _stdev(nums: List[float]) -> float:
    n = len(nums)
    if n <= 1:
        return 0.0
    mean = sum(nums) / n
    var = sum((x - mean) ** 2 for x in nums) / n  # population stdev
    return math.sqrt(var)

@tool
def get_game_stats(group_by: Optional[str] = "Platform",
                   bucket_size: Optional[int] = None) -> Dict[str, Any]:
    """
    Calculate score statistics and distributions for the games dataset.

    args:
        group_by (str | None): Field to group by for a categorical distribution
                               (e.g., "Platform"). Use None to skip grouping.
        bucket_size (int | None): If provided, create a histogram of scores with this
                                  bucket width (e.g., 5 -> 80-84, 85-89, ...).

    returns:
        dict: {
          "count": int,
          "score": {"avg": float, "median": float, "min": int, "max": int, "stdev": float},
          "distribution": {<group_value>: {"count": int, "avg": float}} | null,
          "score_histogram": { "<lo>-<hi>": int } | null
        }
    """
    data = GAMES_DATA
    scores = [row["Score"] for row in data]
    n = len(scores)

    overall = {
        "count": n,
        "score": {
            "avg": sum(scores) / n if n else 0.0,
            "median": _median(scores) if n else 0.0,
            "min": min(scores) if n else 0,
            "max": max(scores) if n else 0,
            "stdev": _stdev(scores) if n else 0.0,
        },
    }

    # Categorical distribution (e.g., by Platform)
    distribution: Optional[Dict[str, Dict[str, float]]] = None
    if group_by:
        buckets: Dict[str, List[int]] = {}
        for row in data:
            key = str(row.get(group_by, "Unknown"))
            buckets.setdefault(key, []).append(row["Score"])
        distribution = {
            k: {"count": len(v), "avg": (sum(v) / len(v) if v else 0.0)}
            for k, v in buckets.items()
        }

    # Score histogram with fixed bucket width
    score_histogram: Optional[Dict[str, int]] = None
    if bucket_size and bucket_size > 0 and n:
        lo = (min(scores) // bucket_size) * bucket_size
        hi = (max(scores) // bucket_size) * bucket_size
        bins: Dict[str, int] = {}
        for s in scores:
            b_lo = (s // bucket_size) * bucket_size
            b_hi = b_lo + (bucket_size - 1)
            label = f"{int(b_lo)}-{int(b_hi)}"
            bins[label] = bins.get(label, 0) + 1
        # Ensure empty bins between lo..hi exist (optional but nice)
        cur = lo
        while cur <= hi:
            label = f"{int(cur)}-{int(cur + bucket_size - 1)}"
            bins.setdefault(label, 0)
            cur += bucket_size
        # Sort labels numerically
        score_histogram = {k: bins[k] for k in sorted(bins.keys(), key=lambda x: int(x.split("-")[0]))}

    overall["distribution"] = distribution
    overall["score_histogram"] = score_histogram
    return overall

In [11]:
# Create an agent with the multiple tools
data_analyst_agent = Agent(
    role="Game Stats Assistant",
    instructions="You can bring insights about a game dataset based on users questions",
    tools=[get_games, get_game_stats]
)

In [12]:
response = data_analyst_agent.invoke("What's the best game in the dataset?")
print(response)

role='system' content="You're an AI Agent and your role is Game Stats AssistantYour instructions: You can bring insights about a game dataset based on users questions"
role='user' content="What's the best game in the dataset?"
role='assistant' content=None tool_calls=[ChatCompletionMessageToolCall(id='call_bIxzE09LBamYdwQkBDAygoa0', function=Function(arguments='{"num_games":1,"top":true}', name='get_games'), type='function')]
role='tool' content='[{"Game": "The Legend of Zelda: Breath of the Wild", "Platform": "Switch", "Score": 98}]' tool_call_id='call_bIxzE09LBamYdwQkBDAygoa0' name='get_games'
role='assistant' content='The best game in the dataset is **The Legend of Zelda: Breath of the Wild** for the **Switch**, with a score of **98**.' tool_calls=None
The best game in the dataset is **The Legend of Zelda: Breath of the Wild** for the **Switch**, with a score of **98**.


In [13]:
response = data_analyst_agent.invoke("What's the worst game in the dataset?")
print(response)

role='system' content="You're an AI Agent and your role is Game Stats AssistantYour instructions: You can bring insights about a game dataset based on users questions"
role='user' content="What's the worst game in the dataset?"
role='assistant' content=None tool_calls=[ChatCompletionMessageToolCall(id='call_PuiQd2HFDngEo0o43dhUIGh7', function=Function(arguments='{"num_games":1,"top":false}', name='get_games'), type='function')]
role='tool' content='[{"Game": "Pikmin 3", "Platform": "Wii U", "Score": 85}]' tool_call_id='call_PuiQd2HFDngEo0o43dhUIGh7' name='get_games'
role='assistant' content='The worst game in the dataset is "Pikmin 3" for the Wii U, with a score of 85.' tool_calls=None
The worst game in the dataset is "Pikmin 3" for the Wii U, with a score of 85.


In [14]:
response = data_analyst_agent.invoke("What is the average score?")
print(response)

role='system' content="You're an AI Agent and your role is Game Stats AssistantYour instructions: You can bring insights about a game dataset based on users questions"
role='user' content='What is the average score?'
role='assistant' content=None tool_calls=[ChatCompletionMessageToolCall(id='call_pIN5uSxxOXHLka73N0wvBooi', function=Function(arguments='{}', name='get_game_stats'), type='function')]
role='tool' content='{"count": 10, "score": {"avg": 91.5, "median": 92.0, "min": 85, "max": 98, "stdev": 4.588027898781785}, "distribution": {"Switch": {"count": 4, "avg": 93.25}, "GameCube": {"count": 1, "avg": 97.0}, "Wii": {"count": 2, "avg": 90.0}, "3DS": {"count": 2, "avg": 90.0}, "Wii U": {"count": 1, "avg": 85.0}}, "score_histogram": null}' tool_call_id='call_pIN5uSxxOXHLka73N0wvBooi' name='get_game_stats'
role='assistant' content='The average score of the games is 91.5.' tool_calls=None
The average score of the games is 91.5.


In [15]:
response = data_analyst_agent.invoke("What is the distribution of platforms?")
print(response)

role='system' content="You're an AI Agent and your role is Game Stats AssistantYour instructions: You can bring insights about a game dataset based on users questions"
role='user' content='What is the distribution of platforms?'
role='assistant' content=None tool_calls=[ChatCompletionMessageToolCall(id='call_TR8YLZLjgeW3Mg70QXD597fr', function=Function(arguments='{"group_by":"Platform"}', name='get_game_stats'), type='function')]
role='tool' content='{"count": 10, "score": {"avg": 91.5, "median": 92.0, "min": 85, "max": 98, "stdev": 4.588027898781785}, "distribution": {"Switch": {"count": 4, "avg": 93.25}, "GameCube": {"count": 1, "avg": 97.0}, "Wii": {"count": 2, "avg": 90.0}, "3DS": {"count": 2, "avg": 90.0}, "Wii U": {"count": 1, "avg": 85.0}}, "score_histogram": null}' tool_call_id='call_TR8YLZLjgeW3Mg70QXD597fr' name='get_game_stats'
role='assistant' content='The distribution of platforms in the dataset is as follows:\n\n- **Switch**: 4 games, average score of 93.25\n- **GameCube*